<a href="https://colab.research.google.com/github/NishantRaj29/Movie-_Recommended_system/blob/main/movie_recommended_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing The dependencies

In [ ]:
import numpy as np
import pandas as pd

import difflib #Closest match in the search
from sklearn.feature_extraction.text import TfidfVectorizer #conversion of character data to numerical data
from sklearn.metrics.pairwise import cosine_similarity


Data Collection and Preprocessing

In [ ]:
movies_data=pd.read_csv('/content/movies.csv')

In [ ]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
movies_data.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [ ]:
#no. of rows and coloumn in the data frame
movies_data.shape

(4803, 24)

In [ ]:
#selecting the relevant feature for feature selection
selected_features=['genres','keywords','tagline','cast','director']

In [ ]:
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
#replacing the null values with null string
for feature in selected_features:
  movies_data[feature]=movies_data[feature].fillna('')

In [ ]:
#combining all the 5 selected feature
combined_features =movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [ ]:
#converting the text data to numerical data
vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors =vectorizer.fit_transform(combined_features)

In [ ]:

print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [ ]:
#getting similiarity scores using cosine similiarity
similarity=cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [ ]:
print(similarity.shape)

(4803, 4803)


In [ ]:
#getting the movie name from the user
movie_name=input('Print the movie name :')

Print the movie name :Money Hiest


In [ ]:
#creating the list with all the movies name given in the dataset

list_of_all_titles=movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
#finding the closest match for the movie name given by the user
find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Money Monster', 'Tower Heist', 'Money Train']


In [ ]:
close_match=find_close_match[0]
print(close_match)

Money Monster


In [ ]:
#finding the index of the movie with title
index_of_the_movie =movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

1770


In [ ]:
#getting the list of similiar movie
similiarity_score=list(enumerate(similarity[index_of_the_movie]))

In [ ]:
print(similiarity_score)

[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.010503132558744178), (4, 0.0), (5, 0.0), (6, 0.03832267328351922), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.005389411410411884), (12, 0.036510348147481504), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.03216724961780326), (24, 0.06187702905383514), (25, 0.009470911765471145), (26, 0.0), (27, 0.005526538619030904), (28, 0.016777210031131126), (29, 0.0063474950860858605), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.006085583812076881), (34, 0.0), (35, 0.01127606090439725), (36, 0.05631397231492762), (37, 0.01125955141242447), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.005844204962652572), (42, 0.0378329815357922), (43, 0.0061668037474148715), (44, 0.0), (45, 0.00853319950792503), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.011755796985022368), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.022711336076174253), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.018993602658816822), (59, 0.0),

In [ ]:
len(similiarity_score)

4803

In [ ]:
#sorting the movie based on the similiarity score
sorted_similiar_score=sorted(similiarity_score,key=lambda x:x[1],reverse=True)
print(sorted_similiar_score)

[(1770, 1.0000000000000002), (2082, 0.18219677721064967), (388, 0.13822889931237134), (780, 0.13804091457660822), (538, 0.1372831624079483), (1688, 0.13520738738397986), (247, 0.13255034876036514), (940, 0.13198880288313575), (1232, 0.12855647821405045), (795, 0.1280334016971147), (4337, 0.12616100595133217), (3347, 0.12560630249145707), (2882, 0.1165724321549521), (1994, 0.11471858563655048), (3619, 0.11463142422394726), (717, 0.11187504652610555), (586, 0.1112918151679423), (3511, 0.10827762286046175), (741, 0.10818018189338027), (3676, 0.10594910253697104), (1010, 0.10522387350418913), (1404, 0.09985755925903479), (222, 0.09776477214123862), (2183, 0.09657207958907134), (386, 0.09411870659521591), (790, 0.09321862440128503), (970, 0.09284123175134137), (751, 0.0923008523388804), (850, 0.09186956550486669), (889, 0.08999806526716096), (2645, 0.08944957200908929), (502, 0.08775298721556118), (825, 0.08628319537159783), (834, 0.08622410245836573), (487, 0.08607398483729584), (233, 0.08

In [ ]:
#print the name of similiar movie based on index

print('Movies suggested for you :\n')

i=1

for movie in sorted_similiar_score:
  index=movie[0]
  title_from_index=movies_data[movies_data.index==index]['title'].values[0]
  if(i<30):
    print(i,'.',title_from_index)
    i=i+1

Movies suggested for you :

1 . Money Monster
2 . The Beaver
3 . Ocean's Eleven
4 . Edtv
5 . Hostage
6 . Confessions of a Dangerous Mind
7 . Ocean's Twelve
8 . Syriana
9 . Morning Glory
10 . Leatherheads
11 . Taxi Driver
12 . Good Night, and Good Luck.
13 . The Dangerous Lives of Altar Boys
14 . The Ides of March
15 . UHF
16 . Jack Reacher
17 . The Monuments Men
18 . Videodrome
19 . Shooter
20 . The Awakening
21 . Panic Room
22 . Street Fighter
23 . Elysium
24 . Home for the Holidays
25 . The Bourne Supremacy
26 . American Sniper
27 . Hannibal Rising
28 . Duplicity
29 . Chain Reaction
